# Data Preprocessing

## Import libraries

In [18]:
import pandas as pd

In [26]:
df = pd.read_csv('data/output/output_dataset.csv')

df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3881,612,16/10/2023,Europe,Virus de la bleuette,"Moutons, vaches et chèvres",médias,Le virus de la bleuette a été détecté en Allem...,Le virus de la bleuette a été trouvé sur une f...,La maladie provoque des dommages économiques s...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3932,605,17/10/2023,Allemagne,Bluetongue,Cattle,médias,Une maladie de loup tigre a été détectée sur u...,"Sur une ferme en Allemagne, une maladie a été ...",Une maladie de loup tigre a été détectée sur u...
2,code153,https://lc.cx/JSB3wp,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18/10/2023,"Irlande du Nord, France, Europe",Bluetongue et EHD,"Bovins, cerfs, moutons",médias,DAERA urge les agriculteurs à être vigilants e...,"La maladie affecte les cerfs, mais des cas cli...",Les experts estiment que les risques principau...
3,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,inconnue,"France, Maroc",nezf oubawi,bateau,médias,France est atteinte par un autre cas de nézfat...,L'Europe est actuellement confrontée à une nou...,Le nézfat oubavi est une maladie animale qui a...
4,code156,https://www.elfagr.org/4789113,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,ar,1962,240,10/19/2023,"Égypte, Province du Gizeh",Taïon,"Agnan, Moutons",médias,Médicins vétérinaires de la province du Gizeh ...,La campagne a été lancée par le gouvernement é...,La campagne de vaccination anti-taïon est une ...


In [27]:
print(df['maladie'])

0                                  Virus de la bleuette
1                                            Bluetongue
2                                     Bluetongue et EHD
3                                           nezf oubawi
4                                                 Taïon
5         Nzef oublotaire (Maladie de la perte de sang)
6                                            حمى قلاعية
7                                    Maladie de tétanos
8                Maladie hémorragique épizootique (MHE)
9                Maladie hémorragique épizootique (MHE)
10               Maladie hémorragique épizootique (MHE)
11                      Dermatose nodulaire contagieuse
12                                   tuberculose bovine
13                      Dermatose nodulaire contagieuse
14                     Maladie hémorragique épizootique
15                                             inconnue
16               MHE (Maladie hémorragique épizootique)
17                                              

In [37]:
# Dictionnaire de regroupement des maladies
mapping = {
    "Virus de la bleuette": "Fièvre catarrhale ovine",
    "Bluetongue": "Fièvre catarrhale ovine",
    "Bluetongue et EHD": "Maladie hémorragique épizootique",
    "nezf oubawi": "Maladie hémorragique épizootique",
    "Taïon": "Maladie hémorragique épizootique",
    "Nzef oublotaire (Maladie de la perte de sang)": "Maladie hémorragique épizootique",
    "حمى قلاعية": "Fièvre aphteuse",
    "Maladie de tétanos": "Tétanos",
    "Maladie hémorragique épizootique (MHE)": "Maladie hémorragique épizootique",
    "Dermatose nodulaire contagieuse": "Dermatose nodulaire contagieuse",
    "tuberculose bovine": "Tuberculose bovine",
    "Maladie hémorragique épizootique": "Maladie hémorragique épizootique",
    "inconnue": "Maladie inconnue",
    "MHE (Maladie hémorragique épizootique)": "Maladie hémorragique épizootique",
    "IQA": "Maladie inconnue",
    "Fièvre de West Nile": "Fièvre de West Nile",
    "Grippe équine (Inf. par le virus de la)": "Influenza équine",
    "SARS-CoV-2 chez les animaux": "Infection SARS-CoV-2 animale",
    "Influenza équine": "Influenza équine",
    "طاعون المجترات الصغيرة": "Peste des Petits Ruminants",
    "Peste des Petits Ruminants (PPR) et Péripneumonie contagieuse": "Peste des Petits Ruminants",
    "Clavelée et variole caprine": "Clavelée / Variole caprine",
    "Dermatose nodulaire contagieuse (Inf. par le virus)": "Dermatose nodulaire contagieuse",
    "Aethina tumida (Petit coléoptère des ruches)": "Petit coléoptère des ruches",
    "Nécrose hématopoïétique infectieuse": "Nécrose hématopoïétique infectieuse",
    "Maladie de l'alsace verte": "Maladie hémorragique épizootique",
    "Fièvre hémorragique de Crimée-Congo": "Fièvre hémorragique Crimée-Congo",
    "infection à virus du Nil occidental": "Fièvre de West Nile",
    "Wester-Nile Virus": "Fièvre de West Nile",
    "Maladie hémorragique épizootique (Inf. par le virus)": "Maladie hémorragique épizootique",
    "Disease unknown": "Maladie inconnue",
    None: "Maladie inconnue",
    float('nan'): "Maladie inconnue"
}

# Nettoyage : créer la colonne malade_clean
df["maladie"] = df["maladie"].replace(mapping)

# # Vérifier un aperçu
# print(df[["maladie", "maladie_clean"]].head())

In [29]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots,maladie_clean
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3881,612,16/10/2023,Europe,Virus de la bleuette,"Moutons, vaches et chèvres",médias,Le virus de la bleuette a été détecté en Allem...,Le virus de la bleuette a été trouvé sur une f...,La maladie provoque des dommages économiques s...,Fièvre catarrhale ovine
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3932,605,17/10/2023,Allemagne,Bluetongue,Cattle,médias,Une maladie de loup tigre a été détectée sur u...,"Sur une ferme en Allemagne, une maladie a été ...",Une maladie de loup tigre a été détectée sur u...,Fièvre catarrhale ovine
2,code153,https://lc.cx/JSB3wp,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18/10/2023,"Irlande du Nord, France, Europe",Bluetongue et EHD,"Bovins, cerfs, moutons",médias,DAERA urge les agriculteurs à être vigilants e...,"La maladie affecte les cerfs, mais des cas cli...",Les experts estiment que les risques principau...,Maladie hémorragique épizootique
3,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,inconnue,"France, Maroc",nezf oubawi,bateau,médias,France est atteinte par un autre cas de nézfat...,L'Europe est actuellement confrontée à une nou...,Le nézfat oubavi est une maladie animale qui a...,Maladie hémorragique épizootique
4,code156,https://www.elfagr.org/4789113,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,ar,1962,240,10/19/2023,"Égypte, Province du Gizeh",Taïon,"Agnan, Moutons",médias,Médicins vétérinaires de la province du Gizeh ...,La campagne a été lancée par le gouvernement é...,La campagne de vaccination anti-taïon est une ...,Maladie hémorragique épizootique


In [23]:
df.columns

Index(['code', 'url', 'titre', 'contenu', 'langue', 'nb_caracteres', 'nb_mots',
       'date_publication', 'lieu', 'maladie', 'animal', 'source_publication',
       'resume_50_mots', 'resume_100_mots', 'resume_150_mots',
       'maladie_clean'],
      dtype='object')

In [ ]:
# from googletrans import Translator

# cols = ['titre', 'contenu', 'lieu', 'maladie', 'animal',
#         'resume_50_mots', 'resume_100_mots', 'resume_150_mots']

# translator = Translator()

# def translate(text):
#     if pd.isna(text):
#         return text
#     try:
#         return translator.translate(text, dest='fr').text
#     except:
#         return text

# # Traduction dans la même colonne (sans créer des colonnes _fr)
# for col in cols:
#     df[col] = df[col].apply(translate)

# print("Traduction terminée vers le français !")

# # Facultatif : sauvegarder
# # df.to_csv("df_translated.csv", index=False)


Traduction terminée vers le français !


In [25]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,...,resume_150_mots,maladie_clean,titre_fr,contenu_fr,lieu_fr,maladie_fr,animal_fr,resume_50_mots_fr,resume_100_mots_fr,resume_150_mots_fr
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3881,612,16/10/2023,Europe,Virus de la bleuette,...,La maladie provoque des dommages économiques s...,Fièvre catarrhale ovine,Le virus de la fièvre catarrhale a été détecté...,Tridge LogoPlateforme de trading mondiale Trid...,Europe,Virus de la bleuette,"Moutons, vaches et chèvres",Le virus de la bleuette a été détecté en Allem...,Le virus de la bleuette a été trouvé sur une f...,La maladie provoque des dommages économiques s...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,3932,605,17/10/2023,Allemagne,Bluetongue,...,Une maladie de loup tigre a été détectée sur u...,Fièvre catarrhale ovine,La fièvre catarrhale officiellement confirmée ...,Logo TridgePlateforme commerciale mondiale Tri...,Allemagne,Fièvre catarrhale,Bétail,Une maladie de loup tigre a été détectée sur u...,"Sur une ferme en Allemagne, une maladie a été ...",Une maladie de loup tigre a été détectée sur u...
2,code153,https://lc.cx/JSB3wp,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4947,770,18/10/2023,"Irlande du Nord, France, Europe",Bluetongue et EHD,...,Les experts estiment que les risques principau...,Maladie hémorragique épizootique,Daera exhorte les agriculteurs à être vigilant...,Dernières nouvelles Boeuf Laitier Mouton Machi...,"Irlande du Nord, France, Europe",Fièvre catarrhale et EHD,"Bovins, cerfs, moutons",DAERA urge les agriculteurs à être vigilants e...,"La maladie affecte les cerfs, mais des cas cli...",Les experts estiment que les risques principau...
3,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,inconnue,"France, Maroc",nezf oubawi,...,Le nézfat oubavi est une maladie animale qui a...,Maladie hémorragique épizootique,économie,"Après la vague de punaises de lit, de nombreux...","France, Maroc",nezf oubawi,bateau,France est atteinte par un autre cas de nézfat...,L'Europe est actuellement confrontée à une nou...,Le nézfat oubavi est une maladie animale qui a...
4,code156,https://www.elfagr.org/4789113,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,بيطري الجيزة: رش وتجريع 6311 رأس ماشية بقوافل ...,ar,1962,240,10/19/2023,"Égypte, Province du Gizeh",Taïon,...,La campagne de vaccination anti-taïon est une ...,Maladie hémorragique épizootique,Gizeh Veterinary : Pulvérisation et dosage de ...,Gizeh Vétérinaire : Pulvérisation et dosage de...,"Égypte, Province du Gizeh",Taïon,"Agnan, Moutons",Médicins vétérinaires de la province du Gizeh ...,La campagne a été lancée par le gouvernement é...,La campagne de vaccination anti-taïon est une ...


In [36]:
from deep_translator import GoogleTranslator
from langdetect import detect

# Colonnes à traduire
cols = [
    'titre', 'contenu', 'lieu', 'maladie', 'animal',
    'resume_50_mots', 'resume_100_mots', 'resume_150_mots'
]

def translate_to_french(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return text
    
    # Détection de langue (si déjà FR → ne traduit pas)
    try:
        if detect(text) == "fr":
            return text
    except:
        pass
    
    # Traduction
    try:
        return GoogleTranslator(source='auto', target='fr').translate(text)
    except:
        return text

# Appliquer traduction dans *les mêmes colonnes*
for col in cols:
    print(f"Traduction de : {col}")
    df[col] = df[col].apply(translate_to_french)

print("✅ Traduction terminée et remplacée dans les mêmes colonnes !")

Traduction de : titre
Traduction de : contenu
Traduction de : lieu
Traduction de : maladie
Traduction de : animal
Traduction de : resume_50_mots
Traduction de : resume_100_mots
Traduction de : resume_150_mots
✅ Traduction terminée et remplacée dans les mêmes colonnes !
